In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['dataset1']


In [2]:
training_path = '../input/dataset1/dataset1/train'
test_path = '../input/dataset1/dataset1/test'
validation_path = '../input/dataset1/dataset1/valid'


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras import optimizers
from keras import regularizers

Using TensorFlow backend.


In [4]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.summary()
classifier.add(Flatten())
# classifier.add(layers.Dropout(0.5))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv

In [5]:
# Step 4 - Full connection
classifier.add(Dense(units = 128, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001), activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 64, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001), activation = 'relu'))
classifier.add(Dropout(0.5))
# classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dense(units = 2, activation = 'softmax'))
#classifier.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.compile(optimizer = optimizers.RMSprop(lr=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
# classifier.compile(optimizer = optimizers.RMSprop(lr=1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
valid_datagen = ImageDataGenerator(rescale = 1./255)

In [7]:
training_set = train_datagen.flow_from_directory(training_path,
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1600 images belonging to 2 classes.


In [8]:
validation_set = train_datagen.flow_from_directory(validation_path,
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 340 images belonging to 2 classes.


In [9]:
history = classifier.fit_generator(
                        training_set,
                        steps_per_epoch=100,
                        epochs=15,
                        validation_data=validation_set,
                        validation_steps=50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
100/100 [==============================] - 41s 410ms/step - loss: 10.0480 - acc: 0.5184 - val_loss: 6.9837 - val_acc: 0.5844
Epoch 2/15
100/100 [==============================] - 33s 333ms/step - loss: 4.8938 - acc: 0.5797 - val_loss: 3.1982 - val_acc: 0.5714
Epoch 3/15
100/100 [==============================] - 33s 331ms/step - loss: 2.3018 - acc: 0.6538 - val_loss: 1.7449 - val_acc: 0.7165
Epoch 4/15
100/100 [==============================] - 33s 326ms/step - loss: 1.5593 - acc: 0.7416 - val_loss: 1.4531 - val_acc: 0.7591
Epoch 5/15
100/100 [==============================] - 32s 323ms/step - loss: 1.3316 - acc: 0.8009 - val_loss: 1.2799 - val_acc: 0.8099
Epoch 6/15
100/100 [==============================] - 32s 322ms/step - loss: 1.1842 - acc: 0.8375 - val_loss: 1.1482 - val_acc: 0.8097
Epoch 7/15
100/100 [==============================] - 33s 333ms/step - loss: 1.0884 - acc: 0.8509 - val_loss: 1.0660 - val_acc: 0.8228
Epoch 

In [10]:
# Epoch 1/2
# 100/100 [==============================] - 149s 1s/step - loss: 0.6284 - acc: 0.6316 - val_loss: 0.5407 - val_acc: 0.7242
# Epoch 2/2
# 100/100 [==============================] - 149s 1s/step - loss: 0.4399 - acc: 0.8031 - val_loss: 0.3633 - val_acc: 0.8539

In [11]:
classifier.save('gender_classifier.h5')

In [12]:
from keras.models import load_model
model = load_model('gender_classifier.h5')
classifier = model

In [13]:
def predict_faces(gender, image_name):
    #Making Single new prediction
    import numpy as np
    #from keras.preprocessing.image import image
    import keras.preprocessing.image as image
    test_image = image.load_img(f'../input/dataset1/dataset1/test/{gender}/{image_name}', target_size = (150, 150))
    test_image = image.img_to_array(test_image)
    #use this function to change the image from 2dimension(64, 64) to 3dimension(64, 64, 3)
    test_image = np.expand_dims(test_image, axis=0)
    #add one more dimension before predicting because the predict method expects a batch
    result = classifier.predict(test_image)

    #training_set.class_indices
    prediction = ''
    if result[0][0] == 1:
        prediction = 'man'
    elif result[0][1] == 1:
        prediction = 'woman'
    print(image_name, prediction)
    return prediction

In [14]:
man_images = os.listdir("../input/dataset1/dataset1/test/man")
woman_images = os.listdir("../input/dataset1/dataset1/test/woman")
res_man = []
res_woman = []
for image_name in man_images:
    res_man.append(predict_faces('man', image_name))
for image_name in woman_images:
    res_woman.append(predict_faces('woman', image_name))
# res

face_7.jpg man
face_39.jpg man
face_154.jpg man
face_153.jpg man
face_178.jpg man
face_152.jpg woman
face_123.jpg man
face_107.jpg man
face_106.jpg man
face_76.jpg woman
face_25.jpg man
face_54.jpg man
face_124.jpg man
face_33.jpg man
face_168.jpg man
face_171.jpg man
face_21.jpg man
face_45.jpg man
face_105.jpg man
face_19.jpg man
face_138.jpg man
face_63.jpg man
face_65.jpg man
face_158.jpg woman
face_180.jpg man
face_97.jpg man
face_182.jpg man
face_167.jpg man
face_44.jpg man
face_159.jpg man
face_165.jpg 
face_50.jpg man
face_139.jpg man
face_175.jpg man
face_71.jpg man
face_10.jpg man
face_11.jpg man
face_96.jpg woman
face_146.jpg woman
face_144.jpg man
face_82.jpg man
face_133.jpg man
face_90.jpg man
face_49.jpg woman
face_118.jpg man
face_1.jpg man
face_2.jpg man
face_74.jpg man
face_181.jpg man
face_95.jpg man
face_93.jpg man
face_40.jpg man
face_53.jpg man
face_101.jpg man
face_67.jpg man
face_174.jpg man
face_27.jpg man
face_29.jpg man
face_12.jpg man
face_43.jpg man
face_13

In [15]:
from collections import Counter
results = (Counter(res_man), Counter(res_woman))
results

(Counter({'man': 145, 'woman': 21, '': 4}),
 Counter({'woman': 155, 'man': 13, '': 2}))

In [16]:
# women
# Counter({'woman': 128, 'man': 39, '': 3}), 75.3% accuracy

# Counter({'woman': 145, 'man': 22, '': 3}), 85.3% accuracy
# Counter({'man': 154, 'woman': 12, '': 4}), 90.60% accuracy

# men